In [2]:
import os
from pathlib import Path
import json
from PIL import Image
import numpy as np
from typing import List
from tqdm import tqdm
import pandas as pd

In [3]:
from os.path import basename, dirname

# Crop family-sample face

In [18]:
FILE_DICT = "file_dict.txt"
BBOX_DICT = "bbox_dict.txt"
DATA_CATEGORY = "aihub_family"

In [5]:
# FILE_DICT = "sample_file_dict.txt"
# BBOX_DICT = "sample_bbox_dict.txt"
# DATA_CATEGORY = "aihub_sample"

## Data dictionary

In [6]:
from fpt.path import DICT, FACE, DTFR

### Open sample_file_dict

In [19]:
with open(DICT / FILE_DICT, "r") as f:
    sample_file_dict = json.load(f)

### Open sample_bbox_dict

In [20]:
with open(DICT / BBOX_DICT, "r") as f:
    sample_bbox_dict = json.load(f)

## Main process

In [21]:
from fpt.path import get_face_target_path
from fpt.data import crop_faces, init_df_sample_face, create_face_series

### Crop faces

In [ ]:
key = "F0001_GM_M_D_75-45-18_07"
value = sample_bbox_dict[key]
image_path = sample_file_dict[key]["image"]
_ = crop_faces(key, value, image_path, visualize=True)

In [25]:
df_sample_face = init_df_sample_face()
for key, value in tqdm(sample_bbox_dict.items()):
    image_path = sample_file_dict[key]["image"]
    out = crop_faces(key, value, image_path)
    classes = [r[0] for r in value]
    for sliced_array, c in zip(out, classes):
        target = get_face_target_path(key, c, category=DATA_CATEGORY)
        face_image = Image.fromarray(np.uint8(sliced_array))
        face_image = face_image.convert('RGB')
        face_image.save(target)
        series = create_face_series(key, c, DATA_CATEGORY, target)
        df_sample_face.loc[target.stem] = series

  1%|          | 615/95148 [02:12<3:58:13,  6.61it/s] 

In [13]:
df_sample_face.to_csv(DTFR / f"df_{DATA_CATEGORY}_face.csv", index_label='uuid')

### Flatten directory

In [14]:
from fpt.data import flatten_directory

In [15]:
flatten_directory(str(FACE), DATA_CATEGORY)

### face image path from series

In [16]:
from fpt.path import get_face_image_path_from_series

In [17]:
get_face_image_path_from_series(df_sample_face.iloc[0])

PosixPath('/home/jongphago/family-photo-tree/data/face-image/aihub_sample/0000/F0001/F0001-GM/34861585-d1f6-4432-8ff7-93b0af5546aa.jpg')